# 🤖 Telegram Video Bot для нарезки видео на шотсы

Этот notebook запускает Telegram бота, который:
- Скачивает видео с YouTube
- Нарезает их на короткие клипы (шотсы)
- Загружает на Google Drive
- Добавляет заголовки и подзаголовки

## 📋 Требования:
1. Токен Telegram бота
2. Google OAuth токен (base64)
3. Репозиторий с кодом на GitHub

## 🔧 Установка зависимостей

In [ ]:
# Установка системных зависимостей
!apt update -qq
!apt install -y ffmpeg

# Проверяем установку ffmpeg
!ffmpeg -version | head -1

In [ ]:
# Клонируем репозиторий с ботом
import os

# Замените на ваш GitHub репозиторий
GITHUB_REPO = "https://github.com/YOUR_USERNAME/YOUR_REPO.git"

# Клонируем репозиторий
if not os.path.exists('telegram-video-bot'):
    !git clone {GITHUB_REPO} telegram-video-bot
else:
    print("Репозиторий уже клонирован")

# Переходим в папку проекта
%cd telegram-video-bot

In [ ]:
# Установка Python зависимостей с оптимизациями для Colab
!pip install -q python-telegram-bot==20.7
!pip install -q yt-dlp==2023.12.30
!pip install -q ffmpeg-python==0.2.0
!pip install -q openai-whisper==20231117
!pip install -q google-api-python-client==2.108.0
!pip install -q google-auth-httplib2==0.1.1
!pip install -q google-auth-oauthlib==1.1.0
!pip install -q python-dotenv==1.0.0
!pip install -q Pillow==10.1.0
!pip install -q moviepy==1.0.3

# Дополнительные пакеты для оптимизации
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q nest-asyncio

print("✅ Все зависимости установлены!")

## ⚙️ Настройка конфигурации

In [ ]:
# Настройка переменных окружения
import os
from google.colab import userdata

# Получаем токены из Colab Secrets или вводим вручную
try:
    # Попытка получить из Colab Secrets
    TELEGRAM_BOT_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')
    GOOGLE_OAUTH_TOKEN_BASE64 = userdata.get('GOOGLE_OAUTH_TOKEN_BASE64')
    print("✅ Токены получены из Colab Secrets")
except:
    # Ввод вручную
    print("❌ Токены не найдены в Colab Secrets")
    print("Введите токены вручную:")
    
    TELEGRAM_BOT_TOKEN = input("Telegram Bot Token: ")
    GOOGLE_OAUTH_TOKEN_BASE64 = input("Google OAuth Token (base64): ")

# Создаем .env файл
env_content = f"""TELEGRAM_BOT_TOKEN={TELEGRAM_BOT_TOKEN}
GOOGLE_OAUTH_TOKEN_BASE64={GOOGLE_OAUTH_TOKEN_BASE64}
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("✅ Файл .env создан")

### 🔐 Как добавить токены в Colab Secrets:

1. Нажмите на иконку 🔑 в левой панели Colab
2. Добавьте следующие секреты:
   - `TELEGRAM_BOT_TOKEN` - токен вашего Telegram бота
   - `GOOGLE_OAUTH_TOKEN_BASE64` - токен Google OAuth в формате base64

Это более безопасный способ хранения токенов!

## 🚀 Запуск бота

In [ ]:
# Проверяем доступные ресурсы Colab
import torch
import psutil
import os

print("🖥️ Информация о системе:")
print(f"CPU ядер: {psutil.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"Свободно RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")

# Проверяем GPU
if torch.cuda.is_available():
    print(f"\n🚀 GPU доступен: {torch.cuda.get_device_name(0)}")
    print(f"CUDA версия: {torch.version.cuda}")
    print(f"GPU память: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
else:
    print("\n⚠️ GPU недоступен, используется CPU")

# Проверяем диск
disk_usage = psutil.disk_usage('/')
print(f"\n💾 Диск: {disk_usage.free / (1024**3):.1f} GB свободно из {disk_usage.total / (1024**3):.1f} GB")

In [ ]:
# Оптимизация настроек для Colab
import os

# Устанавливаем переменные окружения для оптимизации
os.environ['OMP_NUM_THREADS'] = str(min(psutil.cpu_count(), 8))
os.environ['MKL_NUM_THREADS'] = str(min(psutil.cpu_count(), 8))
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Настройки для ffmpeg
os.environ['FFMPEG_THREADS'] = str(min(psutil.cpu_count(), 6))

print("⚡ Настройки оптимизации применены:")
print(f"  - CPU потоки: {os.environ['OMP_NUM_THREADS']}")
print(f"  - FFmpeg потоки: {os.environ['FFMPEG_THREADS']}")
print(f"  - GPU устройство: {os.environ['CUDA_VISIBLE_DEVICES']}")

In [ ]:
# Проверяем импорты
try:
    import telegram
    import yt_dlp
    import ffmpeg
    import whisper
    from google.oauth2.credentials import Credentials
    from googleapiclient.discovery import build
    import moviepy
    from PIL import Image
    import torch
    import concurrent.futures
    print("✅ Все модули импортированы успешно")
    
    # Тест GPU для Whisper
    if torch.cuda.is_available():
        print("🚀 GPU готов для ускорения Whisper")
    else:
        print("⚠️ Whisper будет работать на CPU")
        
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")

In [ ]:
# Запуск бота
import asyncio
import nest_asyncio

# Разрешаем вложенные event loops для Colab
nest_asyncio.apply()

print("🤖 Запускаем Telegram бота...")
print("Для остановки нажмите кнопку 'Interrupt execution' или Ctrl+C")
print("="*50)

# Запускаем бота
!python run_bot.py

## 📱 Как использовать бота:

1. **Запустите бота** - отправьте `/start`
2. **Настройте параметры** (опционально):
   - `/duration 30` - длительность клипов в секундах
   - `/title Мой заголовок` - заголовок для видео
   - `/subtitle Подзаголовок` - подзаголовок для видео
   - `/cookies` - установить cookies для приватных видео
3. **Отправьте видео**:
   - Ссылку на YouTube видео
   - Или загрузите видео файл
4. **Получите результат** - ссылки на обработанные клипы в Google Drive

## 🍪 Работа с Cookies для приватных видео:

Если нужно скачать приватное или ограниченное видео:

1. **Экспорт cookies из браузера**:
   - Установите расширение "Get cookies.txt LOCALLY" для Chrome/Firefox
   - Зайдите на YouTube и авторизуйтесь
   - Нажмите на расширение и скопируйте содержимое cookies

2. **Установка cookies в боте**:
   - Отправьте команду `/cookies`
   - Вставьте скопированное содержимое cookies файла
   - Бот сохранит cookies и сможет скачивать приватные видео

3. **Альтернативные способы получения cookies**:
   - Через DevTools браузера (F12 → Application → Cookies)
   - Экспорт из других расширений для cookies

## 🔧 Возможные проблемы:

- **Ошибка токена**: Проверьте правильность Telegram Bot Token
- **Ошибка Google Drive**: Убедитесь, что OAuth токен действителен
- **Ошибка ffmpeg**: Перезапустите ячейку с установкой системных зависимостей
- **Timeout**: Для длинных видео может потребоваться больше времени
- **Приватное видео**: Используйте команду `/cookies` для установки cookies

## 📊 Мониторинг и логи

In [ ]:
# Проверка логов (если есть)
import os
import glob

# Ищем лог файлы
log_files = glob.glob('*.log')
if log_files:
    print("📋 Найденные лог файлы:")
    for log_file in log_files:
        print(f"  - {log_file}")
        
    # Показываем последние строки из последнего лога
    latest_log = max(log_files, key=os.path.getctime)
    print(f"\n📄 Последние 20 строк из {latest_log}:")
    !tail -20 {latest_log}
else:
    print("📋 Лог файлы не найдены")

In [ ]:
# Мониторинг производительности в реальном времени
import psutil
import torch
import time
from IPython.display import clear_output

def show_system_stats():
    """Показывает текущую статистику системы"""
    clear_output(wait=True)
    
    print("📊 МОНИТОРИНГ ПРОИЗВОДИТЕЛЬНОСТИ")
    print("=" * 50)
    
    # CPU
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"🖥️  CPU: {cpu_percent:.1f}%")
    
    # RAM
    memory = psutil.virtual_memory()
    ram_used = (memory.total - memory.available) / (1024**3)
    ram_total = memory.total / (1024**3)
    ram_percent = memory.percent
    print(f"🧠 RAM: {ram_used:.1f}/{ram_total:.1f} GB ({ram_percent:.1f}%)")
    
    # GPU
    if torch.cuda.is_available():
        gpu_memory_used = torch.cuda.memory_allocated(0) / (1024**3)
        gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        gpu_percent = (gpu_memory_used / gpu_memory_total) * 100
        print(f"🚀 GPU: {gpu_memory_used:.1f}/{gpu_memory_total:.1f} GB ({gpu_percent:.1f}%)")
    else:
        print("🚀 GPU: Недоступен")
    
    # Диск
    disk = psutil.disk_usage('/')
    disk_used = (disk.total - disk.free) / (1024**3)
    disk_total = disk.total / (1024**3)
    disk_percent = (disk_used / disk_total) * 100
    print(f"💾 Диск: {disk_used:.1f}/{disk_total:.1f} GB ({disk_percent:.1f}%)")
    
    # Процессы
    python_processes = [p for p in psutil.process_iter(['pid', 'name', 'cpu_percent', 'memory_percent']) 
                       if 'python' in p.info['name'].lower()]
    
    if python_processes:
        print(f"\n🐍 Python процессы: {len(python_processes)}")
        for proc in python_processes[:3]:  # Показываем топ 3
            print(f"   PID {proc.info['pid']}: CPU {proc.info['cpu_percent']:.1f}%, RAM {proc.info['memory_percent']:.1f}%")
    
    print(f"\n⏰ Обновлено: {time.strftime('%H:%M:%S')}")
    print("\n💡 Для остановки мониторинга нажмите 'Interrupt execution'")

# Запускаем мониторинг (остановите вручную)
try:
    while True:
        show_system_stats()
        time.sleep(5)  # Обновляем каждые 5 секунд
except KeyboardInterrupt:
    print("\n✅ Мониторинг остановлен")

In [ ]:
# Проверка временных файлов
import os

print("📁 Содержимое рабочей директории:")
for item in os.listdir('.'):
    if os.path.isfile(item):
        size = os.path.getsize(item)
        print(f"  📄 {item} ({size} bytes)")
    else:
        print(f"  📁 {item}/")

# Показываем временные папки
temp_dirs = [d for d in os.listdir('.') if d.startswith('temp_')]
if temp_dirs:
    print(f"\n🗂️ Временные папки: {len(temp_dirs)}")
    for temp_dir in temp_dirs[:5]:  # Показываем только первые 5
        files_count = len(os.listdir(temp_dir))
        print(f"  📁 {temp_dir}/ ({files_count} файлов)")

## 🧹 Очистка временных файлов

In [ ]:
# Очистка временных файлов
import os
import shutil
import glob

print("🧹 Очистка временных файлов...")

# Удаляем временные папки
temp_dirs = glob.glob('temp_*')
for temp_dir in temp_dirs:
    if os.path.isdir(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"  🗑️ Удалена папка: {temp_dir}")

# Удаляем временные видео файлы
temp_videos = glob.glob('temp_video_*.mp4') + glob.glob('youtube_video_*.mp4')
for temp_video in temp_videos:
    if os.path.isfile(temp_video):
        os.remove(temp_video)
        print(f"  🗑️ Удален файл: {temp_video}")

print("✅ Очистка завершена!")